In [1]:
import ast
import os
import math
import json
import logging
import warnings
import operator
import subprocess
import time
import datetime as dt
from datetime import date, timedelta, datetime
import pandas as pd
import numpy as np
from pandas.io import gbq
from google.cloud import storage
from pytz import timezone

import geopy.distance
from configparser import ConfigParser, ExtendedInterpolation

# Setup logs
logging.basicConfig(format='%(asctime)s %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [2]:
df_weather = "SELECT * FROM (SELECT Job_Update_Time, timestamp,Extract(Date from Timestamp) as Date1,\
              timestampInit, temp, windSpd, snowfallProb, precipProb, Location from aes-analytics-0002.weathersource_mds.forecast_hour\
              where Location like '%IPL%') where Date1 between DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY) AND DATE_SUB(CURRENT_DATE(),\
              INTERVAL -1 DAY) and Job_Update_Time = (Select max(Job_Update_Time) from\
              (SELECT Job_Update_Time, timestamp,Extract(Date from Timestamp) as Date1, timestampInit,\
              temp, windSpd, snowfallProb, precipProb, Location from aes-analytics-0002.weathersource_mds.forecast_hour\
              where Location like '%IPL%')) order by  location , timestamp desc"
df_weather = gbq.read_gbq(df_weather, project_id='aes-analytics-0002')

In [3]:
logging.info(df_weather.shape)
df_weather.head()

2020-11-23 09:30:50,547 (1440, 9)


,Job_Update_Time,timestamp,Date1,timestampInit,temp,windSpd,snowfallProb,precipProb,Location
0,2020-11-19 07:46:53+00:00,2020-11-24 23:00:00+00:00,2020-11-24,2020-11-19 00:00:00+00:00,42.9,11.7,0.0,50.0,IPL_Marker1
1,2020-11-19 07:46:53+00:00,2020-11-24 22:00:00+00:00,2020-11-24,2020-11-19 00:00:00+00:00,42.0,12.0,0.0,50.0,IPL_Marker1
2,2020-11-19 07:46:53+00:00,2020-11-24 21:00:00+00:00,2020-11-24,2020-11-19 00:00:00+00:00,41.0,12.4,0.0,25.0,IPL_Marker1
3,2020-11-19 07:46:53+00:00,2020-11-24 20:00:00+00:00,2020-11-24,2020-11-19 00:00:00+00:00,41.4,12.3,0.0,25.0,IPL_Marker1
4,2020-11-19 07:46:53+00:00,2020-11-24 19:00:00+00:00,2020-11-24,2020-11-19 00:00:00+00:00,41.9,12.3,0.0,25.0,IPL_Marker1


In [4]:
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'], format = '%Y-%m-%d %H:%M:%S', errors ='coerce')
df_weather['timestamp'] = df_weather['timestamp'].dt.tz_convert('US/Eastern')
df_weather.drop(['Date1'], axis=1, inplace=True)
df_weather['Date'] = df_weather['timestamp'].dt.date

In [5]:
df_weather.sort_values(['timestamp', 'Location'], inplace=True, ascending=False)
df_weather.reset_index(drop=True, inplace=True)

In [6]:
CURRENT_DATE_HOUR = datetime.now(timezone('US/Eastern')).strftime('%Y-%m-%d %H')
# print(CURRENT_DATE_HOUR)
CURRENT_DATE_HOUR = datetime.strptime(CURRENT_DATE_HOUR, '%Y-%m-%d %H')
print(CURRENT_DATE_HOUR)

2020-11-23 04:00:00


In [7]:
six_hours_from_now = CURRENT_DATE_HOUR + timedelta(hours=6)
six_hours_from_now = six_hours_from_now.strftime('%Y-%m-%d %H:%M:%S%z')

twelve_hours_from_now = CURRENT_DATE_HOUR + timedelta(hours=12)
twelve_hours_from_now = twelve_hours_from_now.strftime('%Y-%m-%d %H:%M:%S%z')

twetyfour_hour_from_now = CURRENT_DATE_HOUR + timedelta(hours=24)
twetyfour_hour_from_now = twetyfour_hour_from_now.strftime('%Y-%m-%d %H:%M:%S%z')

logging.info(six_hours_from_now)
logging.info(twelve_hours_from_now)
logging.info(twetyfour_hour_from_now)

2020-11-23 09:30:55,891 2020-11-23 10:00:00
2020-11-23 09:30:55,894 2020-11-23 16:00:00
2020-11-23 09:30:55,895 2020-11-24 04:00:00


In [8]:
CURRENT_DATE_HOUR = CURRENT_DATE_HOUR.strftime('%Y-%m-%d %H:%M:%S')
print(CURRENT_DATE_HOUR)
print(six_hours_from_now)

2020-11-23 04:00:00
2020-11-23 10:00:00


In [9]:
timefilter_list = ['Next 6 Hours', 'Next 12 Hours', 'Next 24 Hours']

next6hours = df_weather[(df_weather['timestamp'] > CURRENT_DATE_HOUR) & (df_weather['timestamp'] <= six_hours_from_now)]
next6hours['timestamp'] = next6hours['timestamp'].astype(str)

next12hours = df_weather[(df_weather['timestamp'] > CURRENT_DATE_HOUR) & (df_weather['timestamp'] <= twelve_hours_from_now)]
next12hours['timestamp'] = next12hours['timestamp'].astype(str)

next24hours = df_weather[(df_weather['timestamp'] > CURRENT_DATE_HOUR) & (df_weather['timestamp'] <= twetyfour_hour_from_now)]
next24hours['timestamp'] = next24hours['timestamp'].astype(str)

/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [10]:
next6hours = list(next6hours['timestamp'].unique())
next6hours

['2020-11-23 10:00:00-05:00',
 '2020-11-23 09:00:00-05:00',
 '2020-11-23 08:00:00-05:00',
 '2020-11-23 07:00:00-05:00',
 '2020-11-23 06:00:00-05:00',
 '2020-11-23 05:00:00-05:00']

In [11]:
next12hours = list(next12hours['timestamp'].unique())
next12hours

['2020-11-23 16:00:00-05:00',
 '2020-11-23 15:00:00-05:00',
 '2020-11-23 14:00:00-05:00',
 '2020-11-23 13:00:00-05:00',
 '2020-11-23 12:00:00-05:00',
 '2020-11-23 11:00:00-05:00',
 '2020-11-23 10:00:00-05:00',
 '2020-11-23 09:00:00-05:00',
 '2020-11-23 08:00:00-05:00',
 '2020-11-23 07:00:00-05:00',
 '2020-11-23 06:00:00-05:00',
 '2020-11-23 05:00:00-05:00']

In [12]:
next24hours = list(next24hours['timestamp'].unique())
next24hours

['2020-11-24 04:00:00-05:00',
 '2020-11-24 03:00:00-05:00',
 '2020-11-24 02:00:00-05:00',
 '2020-11-24 01:00:00-05:00',
 '2020-11-24 00:00:00-05:00',
 '2020-11-23 23:00:00-05:00',
 '2020-11-23 22:00:00-05:00',
 '2020-11-23 21:00:00-05:00',
 '2020-11-23 20:00:00-05:00',
 '2020-11-23 19:00:00-05:00',
 '2020-11-23 18:00:00-05:00',
 '2020-11-23 17:00:00-05:00',
 '2020-11-23 16:00:00-05:00',
 '2020-11-23 15:00:00-05:00',
 '2020-11-23 14:00:00-05:00',
 '2020-11-23 13:00:00-05:00',
 '2020-11-23 12:00:00-05:00',
 '2020-11-23 11:00:00-05:00',
 '2020-11-23 10:00:00-05:00',
 '2020-11-23 09:00:00-05:00',
 '2020-11-23 08:00:00-05:00',
 '2020-11-23 07:00:00-05:00',
 '2020-11-23 06:00:00-05:00',
 '2020-11-23 05:00:00-05:00']

In [13]:
filter_df = pd.DataFrame({'Filter_ID': timefilter_list[0], 'Test_Predicted': next6hours})
filter_df_1 = pd.DataFrame({'Filter_ID': timefilter_list[1], 'Test_Predicted': next12hours})
filter_df_2 = pd.DataFrame({'Filter_ID': timefilter_list[2], 'Test_Predicted': next24hours})

In [14]:
final_df = pd.DataFrame()
final_df = final_df.append([filter_df, filter_df_1, filter_df_2])
final_df.reset_index(drop=True, inplace=True)

## Write to big query

In [17]:
df_weather.to_gbq('mds_outage_restoration.IPL_Weather_Dahsboard', project_id='aes-analytics-0002',
                  chunksize=None, reauth=False, if_exists='replace', auth_local_webserver=False,
                    table_schema=None, location=None, progress_bar=True, credentials=None)

final_df.to_gbq('mds_outage_restoration.IPL_Weather_Timefilter', project_id='aes-analytics-0002',
                    chunksize=None, reauth=False, if_exists='replace', auth_local_webserver=False,
                    table_schema=None, location=None, progress_bar=True, credentials=None)

1440 out of 1440 rows loaded.09:38:12,238 
1it [00:03,  3.56s/it]
42 out of 42 rows loaded.-23 09:38:16,140 
1it [00:02,  3.00s/it]
